In [75]:
import sys
from os import listdir
import os

from xml.dom.minidom import parse
import pandas as pd
import nltk

In [76]:
datadir_drugbank = "/Users/lluccardoner/PycharmProjects/MET_AHLT_Lab_2019/data/Train/DrugBank/"
datadir_medline = "/Users/lluccardoner/PycharmProjects/MET_AHLT_Lab_2019/data/Train/MedLine/"

# Load data

In [77]:
def parse_XML_file(datadir):
    trees = []
    for f in listdir(datadir):
        # parse XML file
        tree = parse(datadir + "/" + f)
        trees.append(tree)
    return trees

def parsed_to_df(parsed):
    items = []
    for tree in parsed:
        for s in tree.getElementsByTagName("sentence"):
            s_id = s.attributes["id"].value
            s_txt = s.attributes["text"].value
            for e in s.getElementsByTagName("entity"):
                e_id = e.attributes["id"].value
                e_offset = e.attributes["charOffset"].value 
                e_name = e.attributes["text"].value
                e_type = e.attributes["type"].value
                item = {"s_id": s_id, "s_txt": s_txt, "e_id": e_id, "e_offset": e_offset, "e_name": e_name, "e_type": e_type}
                items.append(item)
                
    return pd.DataFrame(items)

def load_data(datadir, training_set):
    trees = parse_XML_file(datadir)
    df_loaded = parsed_to_df(trees)
    df_loaded['training_set'] = training_set
    return df_loaded

In [78]:
df_drugbank = load_data(datadir_drugbank, "drugbank")
df_drugbank.shape

(12929, 7)

In [79]:
df_medline = load_data(datadir_medline, "medline")
df_medline.shape

(1836, 7)

In [80]:
df = df_drugbank.append(df_medline, ignore_index=True)
df.shape

(14765, 7)

## Entities with sentences

In [81]:
df_drugbank.head()

,e_id,e_name,e_offset,e_type,s_id,s_txt,training_set
0,DDI-DrugBank.d481.s0.e0,calcium,25-31,drug,DDI-DrugBank.d481.s0,"Milk, milk products, and calcium-rich foods or...",drugbank
1,DDI-DrugBank.d481.s0.e1,EMCYT,82-86,brand,DDI-DrugBank.d481.s0,"Milk, milk products, and calcium-rich foods or...",drugbank
2,DDI-DrugBank.d419.s0.e0,allopurinol,33-43,drug,DDI-DrugBank.d419.s0,The concurrent administration of allopurinol a...,drugbank
3,DDI-DrugBank.d419.s0.e1,ampicillin,49-58,drug,DDI-DrugBank.d419.s0,The concurrent administration of allopurinol a...,drugbank
4,DDI-DrugBank.d419.s0.e2,ampicillin,175-184,drug,DDI-DrugBank.d419.s0,The concurrent administration of allopurinol a...,drugbank


In [82]:
df_medline.head()

,e_id,e_name,e_offset,e_type,s_id,s_txt,training_set
0,DDI-MedLine.d69.s0.e0,contortrostatin,70-84,drug_n,DDI-MedLine.d69.s0,Differential regulation of tyrosine phosphoryl...,medline
1,DDI-MedLine.d69.s0.e1,echistatin,141-150,drug_n,DDI-MedLine.d69.s0,Differential regulation of tyrosine phosphoryl...,medline
2,DDI-MedLine.d69.s0.e2,flavoridin,156-165,drug_n,DDI-MedLine.d69.s0,Differential regulation of tyrosine phosphoryl...,medline
3,DDI-MedLine.d69.s1.e0,contortrostatin,28-42,drug_n,DDI-MedLine.d69.s1,The homodimeric disintegrin contortrostatin wa...,medline
4,DDI-MedLine.d69.s1.e1,echistatin,96-105,drug_n,DDI-MedLine.d69.s1,The homodimeric disintegrin contortrostatin wa...,medline


### Total loaded rows

In [83]:
df[['e_type', 'training_set']].groupby(["training_set"]).count()

,e_type
training_set,
drugbank,12929
medline,1836


In [84]:
df[['e_type', 'training_set']].groupby(["training_set"])['e_type'].value_counts()

training_set  e_type
drugbank      drug      8197
              group     3206
              brand     1423
              drug_n     103
medline       drug      1228
              drug_n     401
              group      193
              brand       14
Name: e_type, dtype: int64

In [85]:
df[['e_name', 'training_set']].groupby(["training_set"])["e_name"].value_counts()

training_set  e_name                                                                      
drugbank      digoxin                                                                         176
              warfarin                                                                        176
              phenytoin                                                                       157
              lithium                                                                         119
              theophylline                                                                    115
              ketoconazole                                                                    107
              cimetidine                                                                       89
              carbamazepine                                                                    81
              aspirin                                                                          79
              erythromycin 

In [86]:
df.groupby(["training_set"]).describe()

e_id                                      e_name         \
              count unique                      top freq  count unique   
training_set                                                             
drugbank      12929  12929  DDI-DrugBank.d549.s3.e2    1  12929   2655   
medline        1836   1836    DDI-MedLine.d63.s8.e0    1   1836    556   

                           e_offset        ...  e_type         s_id         \
                  top freq    count unique ...     top  freq  count unique   
training_set                               ...                               
drugbank      digoxin  176    12929   3957 ...    drug  8197  12929   4683   
medline       digoxin   33     1836   1206 ...    drug  1228   1836    877   

                                         s_txt         \
                               top freq  count unique   
training_set                                            
drugbank      DDI-DrugBank.d64.s87   55  12929   4505   
medline        DDI-MedLine.d111.s5   14   1836    877   

                                                                      
                                                            top freq  
training_set                                                          
drugbank      Drugs that reportedly may increase oral antico...   55  
medline       acetaminophen/theophylline, lidocaine/quinidin...   14  

[2 rows x 24 columns]

## Entities and type 

In [87]:
df_e = df[["e_name", "e_type", "training_set"]].drop_duplicates()
df_e.head(10)

,e_name,e_type,training_set
0,calcium,drug,drugbank
1,EMCYT,brand,drugbank
2,allopurinol,drug,drugbank
3,ampicillin,drug,drugbank
7,AUGMENTIN XR,brand,drugbank
12,broad-spectrum antibiotics,group,drugbank
14,contraceptives,group,drugbank
15,PROCRIT,brand,drugbank
16,central nervous system depressants,group,drugbank
17,benzodiazepines,group,drugbank


In [88]:
# there are names that have more than one type
df_e.groupby(["training_set"]).describe()

e_name                    e_type                   
              count unique    top freq  count unique   top  freq
training_set                                                    
drugbank       2719   2655  ergot    2   2719      4  drug  1393
medline         560    556   ACTH    2    560      4  drug   306

In [89]:
df[df["e_name"]=="ACTH"].head()

,e_id,e_name,e_offset,e_type,s_id,s_txt,training_set
4156,DDI-DrugBank.d318.s2.e2,ACTH,35-38,group,DDI-DrugBank.d318.s2,Corticosteroids and Corticotropin (ACTH): may ...,drugbank
9155,DDI-DrugBank.d162.s6.e1,ACTH,17-20,drug,DDI-DrugBank.d162.s6,"Corticosteroids, ACTH: intensified electrolyte...",drugbank
9628,DDI-DrugBank.d17.s9.e3,ACTH,52-55,drug,DDI-DrugBank.d17.s9,Amphotericin B or Corticosteroids or Corticotr...,drugbank
10913,DDI-DrugBank.d46.s9.e1,ACTH,19-22,drug,DDI-DrugBank.d46.s9,"- Corticosteroids, ACTH: Intensified electroly...",drugbank
11721,DDI-DrugBank.d10.s1.e1,ACTH,111-114,drug,DDI-DrugBank.d10.s1,Although studies designed to examine drug inte...,drugbank


## Feature analysis

### Upper case

In [90]:
df_e['is_upper'] = df_e['e_name'].apply(lambda x : x.isupper())
df_e.head()

,e_name,e_type,training_set,is_upper
0,calcium,drug,drugbank,False
1,EMCYT,brand,drugbank,True
2,allopurinol,drug,drugbank,False
3,ampicillin,drug,drugbank,False
7,AUGMENTIN XR,brand,drugbank,True


In [91]:
df_e.groupby(['training_set', 'e_type', 'is_upper']).count()

e_name
training_set e_type is_upper        
drugbank     brand  False        225
                    True         165
             drug   False       1356
                    True          37
             drug_n False         57
                    True           9
             group  False        852
                    True          18
medline      brand  False          6
             drug   False        296
                    True          10
             drug_n False        101
                    True          22
             group  False        120
                    True           5

### Suffixes

In [92]:
df_e['suffix_5'] = df_e['e_name'].apply(lambda x : x[-5:].lower())
df_e.head()

,e_name,e_type,training_set,is_upper,suffix_5
0,calcium,drug,drugbank,False,lcium
1,EMCYT,brand,drugbank,True,emcyt
2,allopurinol,drug,drugbank,False,rinol
3,ampicillin,drug,drugbank,False,illin
7,AUGMENTIN XR,brand,drugbank,True,in xr


In [98]:
initial_drug_suffixes = ["idase", "idone", "uride", "ogens", "rinol", "amate", "lones", "pamil", "olone", "parin", "ssant", "udine",
            "D", "etron", "adiol", "feine", "pines", "zines", "toxin", "MAO", "opram", "ophen", "sides", "talis",
            "ulant", "ylate", "osine", "oxide", "caine", "illin", "itant", "limus", "pride", "sulin", "oride", "abine",
            "hrine", "iazem", "atory", "oidal", "emide", "hanol", "phine", "SAIDs", "coxib", "necid", "nists", "esium",
            "acids", "nolol", "nafil", "azine", "exate", "rates", "cking", "lcium", "azide", "zepam", "arone", "rofen",
            "ampin", "ergic", "roids", "tamin", "adine", "odium", "bital", "pirin", "lants", "orine", "mines", "zolam",
            "apine", "cohol", "ckers", "ipine", "acid", "yclic", "otics", "tives", "xacin", "etine", "epine", "drugs",
            "tatin", "thium", "lline", "amide", "sants", "etics", "itors", "ytoin", "gents", "navir", "goxin", "farin",
            "mycin", "idine", "amine", "azole"]
print(len(initial_drug_suffixes), len(new_type_suffix_5["drug"]))

100 411


In [99]:
len(intersection(initial_drug_suffixes, new_type_suffix_5["drug"]))

34

In [100]:
#for k in new_type_suffix_5:
#    with open(k+'_suffix_5_DrugBank.txt', 'w') as f:
#            for item in new_type_suffix_5[k]:
#                f.write("%s\n" % item)

In [101]:
#loaded = {}
#for k in new_type_suffix_5:
#    with open(k+'_suffix_5_DrugBank.txt', 'r') as f:
#        loaded[k] = f.read().splitlines()

In [102]:
# df_e[df_e["e_name"].str.contains("propranolo")]

,e_name,e_type,training_set,is_upper,suffix_5
1045,propranolol,drug,drugbank,False,nolol
14704,propranolol,drug,medline,False,nolol


### Plural

In [103]:
df_e['is_plural'] = df_e['e_name'].apply(lambda x : x[-1:] == 's')
df_e.head(10)

,e_name,e_type,training_set,is_upper,suffix_5,is_plural
0,calcium,drug,drugbank,False,lcium,False
1,EMCYT,brand,drugbank,True,emcyt,False
2,allopurinol,drug,drugbank,False,rinol,False
3,ampicillin,drug,drugbank,False,illin,False
7,AUGMENTIN XR,brand,drugbank,True,in xr,False
12,broad-spectrum antibiotics,group,drugbank,False,otics,True
14,contraceptives,group,drugbank,False,tives,True
15,PROCRIT,brand,drugbank,True,ocrit,False
16,central nervous system depressants,group,drugbank,False,sants,True
17,benzodiazepines,group,drugbank,False,pines,True


In [104]:
df_e.groupby(['training_set', "e_type", "is_plural"]).count()

e_name  is_upper  suffix_5
training_set e_type is_plural                            
drugbank     brand  False         383       383       383
                    True            7         7         7
             drug   False        1355      1355      1355
                    True           38        38        38
             drug_n False          64        64        64
                    True            2         2         2
             group  False         227       227       227
                    True          643       643       643
medline      brand  False           6         6         6
             drug   False         294       294       294
                    True           12        12        12
             drug_n False         116       116       116
                    True            7         7         7
             group  False          46        46        46
                    True           79        79        79

### Part Of Speech

In [105]:
df_e['e_pos'] = df_e['e_name'].apply(lambda x : nltk.pos_tag([x])[0][1])
df_e.head(10)

,e_name,e_type,training_set,is_upper,suffix_5,is_plural,e_pos
0,calcium,drug,drugbank,False,lcium,False,NN
1,EMCYT,brand,drugbank,True,emcyt,False,NN
2,allopurinol,drug,drugbank,False,rinol,False,NN
3,ampicillin,drug,drugbank,False,illin,False,NN
7,AUGMENTIN XR,brand,drugbank,True,in xr,False,NN
12,broad-spectrum antibiotics,group,drugbank,False,otics,True,NNS
14,contraceptives,group,drugbank,False,tives,True,NNS
15,PROCRIT,brand,drugbank,True,ocrit,False,NN
16,central nervous system depressants,group,drugbank,False,sants,True,NNS
17,benzodiazepines,group,drugbank,False,pines,True,NNS


In [106]:
df_e.groupby(['training_set', "e_type", "e_pos"]).count()

e_name  is_upper  suffix_5  is_plural
training_set e_type e_pos                                       
drugbank     brand  IN          2         2         2          2
                    JJ          9         9         9          9
                    NN        354       354       354        354
                    NNP        18        18        18         18
                    NNS         4         4         4          4
                    RB          1         1         1          1
                    VB          1         1         1          1
                    VBN         1         1         1          1
             drug   IN          3         3         3          3
                    JJ         30        30        30         30
                    NN       1291      1291      1291       1291
                    NNP        12        12        12         12
                    NNS        38        38        38         38
                    RB         16        16        16         16
                    VB          2         2         2          2
                    VBG         1         1         1          1
             drug_n CD          1         1         1          1
                    JJ         11        11        11         11
                    NN         52        52        52         52
                    NNS         2         2         2          2
             group  CD          1         1         1          1
                    JJ         37        37        37         37
                    NN        201       201       201        201
                    NNP         2         2         2          2
                    NNS       629       629       629        629
medline      brand  NN          6         6         6          6
             drug   CD          2         2         2          2
                    JJ          5         5         5          5
                    NN        287       287       287        287
                    NNP         4         4         4          4
                    NNS         7         7         7          7
                    VB          1         1         1          1
             drug_n CD          1         1         1          1
                    JJ         13        13        13         13
                    NN        101       101       101        101
                    NNP         1         1         1          1
                    NNS         5         5         5          5
                    VB          1         1         1          1
                    VBN         1         1         1          1
             group  JJ         10        10        10         10
                    NN         40        40        40         40
                    NNS        75        75        75         75